# Preprocessing for Transformers
### Done 
[1] Remove NaN <br>
[2] rename Labels and Text<br>
[3] rename sentiment 1 (negative) and 0(positive) (and 2 neutral)<br>

In this notebook, text without preprocessing is loaded with the ids from the preprocessed corpora which are used in the svm classification task. Smaller corpora are getting transformed in the "Preprocessing_for_Transformers_Small_Files" notebook, because of performance issues. <br>

### Information about the different functions used here: <br>
[1] **loadFiles:** <br>
All paths of the different corpora are stored in lists here <br>
[2] **extractText:** <br>
Get the text data from the raw data corpus with the ids  <br>
[3] **insertNewData:** <br>
Insert the new text data in a dataframe and transfer this to a list. Also add the data to the result dataframe <br>
[4] **updateDataframe:** <br>
Updates the dataframe, changes column names and replace label names <br>
[5] **main:** <br>
Loads, transform and save the data <br>

In [17]:
import pandas as pd
import numpy as np
import os
import re

In [18]:
def loadFiles():
    file_binary=["../../Corpora/Preprocessed/Binary/RE02_scare_Preprocessed_binary_balanced.tsv",
        "../../Corpora/Preprocessed/Binary/RE04_filmstarts_Preprocessed_binary.tsv",
        "../../Corpora/Preprocessed/Binary/RE05_amazonreviews_Preprocessed_binary_balanced.tsv"]

    file_binary_raw=["../../Corpora/Normalized/RE02_scare_all.tsv",
        "../../Corpora/Normalized/RE04_filmstarts_all.tsv",
        "../../Corpora/Normalized/RE05_amazonreviews_all.tsv"]

    file_ternary=["../../Corpora/Preprocessed/Ternary/RE02_scare_Preprocessed_ternary_balanced.tsv",
        "../../Corpora/Preprocessed/Ternary/RE04_filmstarts_Preprocessed_ternary_all.tsv"]

    file_ternary_raw=["../../Corpora/Normalized/RE02_scare_all.tsv",
        "../../Corpora/Normalized/RE04_filmstarts_all.tsv"]
    return file_ternary_raw, file_ternary, file_binary_raw, file_binary

In [19]:
def updateDataframe(resultDf):
    resultDf=resultDf.rename(columns={"sentiment":"labels"})
    resultDf["labels"].replace(to_replace="positive", value=0, regex=True, inplace=True)
    resultDf["labels"].replace(to_replace="negative", value=1, regex=True, inplace=True)
    resultDf["labels"].replace(to_replace="neutral", value=2, regex=True, inplace=True)
    return resultDf

In [20]:
def insertNewData(updateTextDf,resultDf):
    textList=[]
    flatTextList=[]
    textList=updateTextDf.values.tolist()
    for item in textList:
        for subitem in item:
            flatTextList.append(subitem)

    resultDf["text"]=flatTextList
    return resultDf

In [21]:
def extractText(resultDf,textDf):
    allDataList=[]
    for ids in range(len(resultDf["id"])):
        number=resultDf["id"][ids]
        allDataList.append(textDf.iloc[number])
            
    return allDataList

In [22]:
def main():
    loadingData=loadFiles()
    file_ternary_raw=loadingData[0]
    file_ternary=loadingData[1]
    file_binary_raw=loadingData[2]
    file_binary=loadingData[3]
    
    for index in range(len(file_ternary_raw)):
        print("Start")
        print("Index of List: ",index)
        
        file = file_ternary[index]
        fileText = file_ternary_raw[index]
        
        path,fileName =os.path.split(file)
        fileName= re.sub(".tsv","",fileName)
        
        print("Name of Corpus: ", fileName)

        idDf = pd.read_csv(file , sep="\t")
        textDf=pd.read_csv(fileText,sep="\t")
        
        resultDf= pd.DataFrame()
        
        resultDf=idDf.filter(["id","sentiment"])
        textDf=textDf.filter(["id","text","sentiment"])

        allDataList=extractText(resultDf,textDf)
        
        updateTextDf = pd.DataFrame(columns=["id","text","sentiment"])
        updateTextDf = pd.DataFrame(allDataList,columns=["id","text","sentiment"])
        updateTextDf=updateTextDf.filter(["text"])
        
        resultDf=insertNewData(updateTextDf,resultDf)
        resultDf=updateDataframe(resultDf)

        print("New Dataframe: ",resultDf)
        print("-------------------------")
        
        newFileName=fileName+"_Transformer.tsv"
        resultDf.to_csv(newFileName, sep="\t",index=False)
main()

Start
Index of List:  0
Name of Corpus:  RE02_scare_Preprocessed_ternary_balanced
New Dataframe:             id  labels                                               text
0      262230       0         Super Spiel....nur leider sehr donutlastig
1       88634       0                        Das Programm ist sehr gut .
2      171339       0  Eigentlich ist es das beste Navi. Deswegen hab...
3      123422       0  Man bekommt auf die sekunde genau eine Nachric...
4        1100       0  Sehr gut! Läuft bei mir problemlos aus Samsung...
...       ...     ...                                                ...
69997  553605       2  Bei mir kommt immer Werbung von Bild irgendwo ...
69998  636193       2  bringt leider nicht die erhoffte Verbesserung....
69999  142586       2           Kann seit kurzem keine Fotos runterladen
70000   30092       2  Ich finde freeletics generell extrem gut, woll...
70001  559598       2  ... bis das letzte Update kam! Dieses ließ sic...

[70002 rows x 3 columns]
